In [1]:
import json

with open("datasets/intents.json") as file:
  data = json.load(file)

type(data)

dict

In [ ]:
import nltk

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("omw-1.4")

##### Stemming vs Lemmatization

* Stemming is a process that stems or removes last few characters from a word, often leading to incorrect meanings and spelling.
* Lemmatization considers the context and converts the word to its meaningful base form, which is called Lemma.

In [3]:
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

words = ["babies", "women", "children", "feet", "libraries"]

for word in words:
  print(word, "=>", wnl.lemmatize(word))

babies => baby
women => woman
children => child
feet => foot
libraries => library


In [4]:
from nltk.tokenize import word_tokenize

vocabularies = []
classes = []
doc_x = [] # questions
doc_y = [] # question classes

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = word_tokenize(pattern)
    vocabularies.extend(tokens)
    doc_x.append(pattern)
    doc_y.append(intent["class"])

  if intent["class"] not in classes:
    classes.append(intent["class"])

In [5]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")

In [6]:
import string

vocabularies = [word for word in vocabularies if word not in string.punctuation]

vocabularies = [wnl.lemmatize(word.lower()) for word in vocabularies]

vocabularies = [word for word in vocabularies if word not in stop_words]

vocabularies = set(vocabularies)

vocabularies = sorted(vocabularies)

vocabularies

['account',
 'amenity',
 'anyone',
 'appreciate',
 'assistance',
 'availability',
 'available',
 'birthday',
 'book',
 'booking',
 'born',
 'bye',
 'call',
 'cancel',
 'check',
 'complaint',
 'could',
 'create',
 'date',
 'get',
 'give',
 'goodbye',
 'hand',
 'hang',
 'hay',
 'hello',
 'help',
 'helpful',
 'hey',
 'hi',
 'hotel',
 'later',
 'make',
 'name',
 'need',
 'new',
 'ocean',
 'offer',
 'old',
 'open',
 'option',
 'overlooking',
 'plan',
 'please',
 'pool',
 'raise',
 'reservation',
 'reserve',
 'room',
 'sea',
 'see',
 'service',
 'support',
 'swimming',
 'tell',
 'thank',
 'thanks',
 'time',
 'type',
 'view',
 'wa',
 'want',
 'week',
 'weekend',
 'whats']

In [7]:
classes = sorted(set(classes))

classes

['about',
 'age',
 'amenities',
 'cancel_reservation',
 'check_availability',
 'complaint',
 'createaccount',
 'date',
 'goodbye',
 'greeting',
 'help',
 'name',
 'pool',
 'reservation',
 'room_types',
 'sea_view',
 'services',
 'thanks']

In [36]:
x = []
y = []

for idx, doc in enumerate(doc_x):
  bow = [] # bag of words

  doc = wnl.lemmatize(doc.lower())
  
  for word in vocabularies:
    bow.append(1 if word in doc else 0)

  x.append(bow)

  ohe = [0] * len(classes) # one hot encoding
  intent_class = doc_y[idx]
  class_index = classes.index(intent_class)
  ohe[class_index] = 1

  y.append(ohe)

In [37]:
import numpy as np

x = np.array(x)

print(doc_x[12])
print(x[12])

Can I check room availability?
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [38]:
y = np.array(y)

print(doc_y[12])
print(y[12])

check_availability
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [39]:
from keras import Sequential
from keras.layers import Input, Dense, Dropout

input_shape = (len(x[0]),) # vocabularies
output_shape = len(y[0]) # classes


output_shape = len(y[0])
model = Sequential([
  Input(shape = input_shape),
  Dense(128, activation = "relu"),
  Dropout(0.5),
  Dense(64, activation = "relu"),
  Dropout(0.3),
  Dense(output_shape, activation = "softmax"),
])

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [40]:
model_ouput = model.fit(x, y, epochs = 200)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0938 - loss: 2.8853
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0312 - loss: 2.9010 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0312 - loss: 2.8886     
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1042 - loss: 2.8837 
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1354 - loss: 2.8343 
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0729 - loss: 2.8421 
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0938 - loss: 2.8038 
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2292 - loss: 2.7614 
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1250 - loss: 2.7876
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1250 - loss: 2.7731  
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1667 - loss: 2.7831  
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2396 

In [41]:
model.save("24-dumps/model.keras")

In [ ]:
from joblib import dump

dump(vocabularies, "24-dumps/vocabularies.pkl")
dump(classes, "24-dumps/classes.pkl")

In [87]:
def process_text(text):
  text = text.lower()

  words = word_tokenize(text)
  words = [word for word in words if word not in string.punctuation]
  words = [wnl.lemmatize(word.lower()) for word in words]
  words = [word for word in words if word not in stop_words]
  words = set(words)
  words = sorted(words)

  bow = [] # bag of words
  for word in vocabularies:
    bow.append(1 if word in words else 0)

  return np.array([bow])

In [58]:
def predict_class(text):
  bow = process_text(text)
  predictions = model.predict(bow, verbose = 0)

  predicted_class_index = np.argmax(predictions[0])

  return classes[predicted_class_index]

In [81]:
import random

def get_response(intent_class, intents):
  intent = next(intent for intent in intents if intent.get("class") == intent_class)

  return random.choice(intent["responses"])

In [93]:
def chatbot():
  text = input("")

  while len(text) > 0:
    predicted_class = predict_class(text)
    response = get_response(predicted_class, data["intents"])
    print(f"{text}\n")
    print(f"> {response}\n")

    text = input("")

In [94]:
chatbot()

Hello!

> Hello! Please let me help you

What are room types do you have?

> We offer various types of rooms. Here are the available options:

1. Standard Room:
Price: $100 per night
Description: A comfortable room with essential amenities.

2. Deluxe Room:
Price: $150 per night
Description: A spacious and well-appointed room with additional amenities.

3. Suite:
Price: $200 per night
Description: A luxurious suite with separate living and sleeping areas, ideal for those seeking extra comfort and space.

Please let me know your preferred room type.

Do you have a pool?

> Yes, we have a beautiful swimming pool available for our guests.
It's perfect for relaxation and enjoyment during your stay.
Please let me know if you'd like more information about pool timings or any other details.

